# Description
* Author: Seydou DIA & Simona MILADINOVA

Notebook to compute metrics.

# Setup

In [ ]:
%matplotlib notebook
%load_ext autoreload

## Imports

In [ ]:
# os related
from pathlib import Path
import os 

# data related
import pandas as pd
# import missingno as msno
import datetime as dt
import numpy as np


# visual related
import matplotlib.pyplot as plt
# time related

from fonctions import *

## Paths

In [ ]:

# MUST BE IN CODE FOLDER TO WORK !
PROJECT_PATH = Path(os.getcwd()).parent

RAW_DATA = PROJECT_PATH / "RAW_DATA" 
PROC_DATA = PROJECT_PATH / "PROC_DATA"

# RAW FILES
CSV_FILE = RAW_DATA / "Individual_stock_data_csv.csv"
XLSX_FILE = RAW_DATA / "Individual_stock_data.xlsx"

# PROC FILES
PROC_STOCK = PROC_DATA / "proc_stock.csv"
PROC_CARBON = PROC_DATA / "proc_carbon.csv"

STOCK_2000 = PROC_DATA / "stock_2000.csv"

STOCK_WORK = PROC_DATA / "stock_work.csv"
STOCK2USE = PROC_DATA / "stock2use.csv"
DIC2USE = PROC_DATA / "dic2use.json"


OUT_RENDEMENT = PROC_DATA / "rendement_v1.csv"

OUT_DF_CSV = PROC_DATA / "optimize_assets.csv"

# OUT_DIC_WEIGHT = PROC_DATA / "dic_weight.pickle"

## Utils

In [ ]:
def compute_enc(weights):
    """Function that computes the ENC for given weights an a given window"""
    return 1 / sum([w**2 for w in weights])
    

def compute_ir(excess_return, track_err):
    """Function that computes the information ratio given 
    the excess return and the track_err"""
    return [ex_ret / track_err for (ex_ret, track_err) in zip(excess_return, track_err)]

def compute_di(weight, stock, vol_p):
    """Function that computes the diversification index"""
    stock_vol = stock.std() * np.sqrt(12)
    return np.dot(stock_vol, weight) / (vol_p * np.sqrt(12))

# Notebook


In [ ]:
stock_df = read_file(STOCK2USE).iloc[104:-4]

return_df = pd.read_csv(OUT_DF_CSV)
return_df.index = read_file(STOCK2USE).index[104:-4]
return_df.drop(columns="Unnamed: 0", inplace=True)

stock_ret = stock_df.pct_change()

sp500 = read_file(STOCK_WORK)
bm_return = equally_weighted(sp500) 

return_sp = pd.DataFrame((sp500.mean(axis=1).pct_change())*100, columns=["mu"])
return_sp = return_sp.loc[return_df.index[0]:return_df.index[-1]]

In [ ]:

metrics = ["ret", "vol", "sr", 
           "track_err", "yearly_track_error",
           "ex_ret", "enc", "encb", 
           "ir", "di"]


In [ ]:
dic = dict.fromkeys(return_df.columns)
for k in dic.keys():
    dic[k] = dict.fromkeys(metrics)


In [ ]:
# Poids pour chacun des rebalancement
dic_weight = {
    pf: pd.read_csv(PROC_DATA / f"weight_{pf}.csv").rename(columns={'Unnamed: 0':"quarter"}).set_index("quarter")
    for pf in dic.keys()
}

In [ ]:
rf = 0.05

In [ ]:
for pf in dic.keys():
        
    # Rendement annualisé
    dic[pf]["ret"] = return_df[pf].mean() * 52

    # Volatilité sur les 18 ans
    dic[pf]["vol"] = return_df[pf].std() * np.sqrt(52)
    
    # Ratio de sharp
    dic[pf]["sr"] = (dic[pf]["ret"] - rf) / dic[pf]["vol"]
    
     # Tracking error
    dic[pf]["track_err"] = []
    for i in range(len(return_df)//12): # par paquet de 3 mois
        dic[pf]["track_err"].append((return_df[pf].iloc[i*12:(i+1)*12] - return_sp["mu"].iloc[i*12:(i+1)*12]).std() * np.sqrt(52))
    
    # Tracking global error
    dic[pf]["yearly_track_err"] = []
    for i in range(len(return_df)//(12*4)): # par paquet de 1 an
        dic[pf]["yearly_track_err"].append((return_df[pf].iloc[i*12*4:(i+1)*12*4] - return_sp["mu"].iloc[i*12*4:(i+1)*12*4]).std() * np.sqrt(52))
    
    # Excess return 
    dic[pf]["ex_ret"] = []
    for i in range(len(return_df)//12): # par paquet de 3 mois
        dic[pf]["ex_ret"].append( 
            52 * (return_df[pf].iloc[i*12:(i+1)*12].mean()) - 52 * (return_sp["mu"].iloc[i*12:(i+1)*12].mean())
        )
        
    # Yearly Excess return 
    dic[pf]["yearly_ex_ret"] = []
    for i in range(len(return_df)//(12*4)): # par paquet de 1 an
        dic[pf]["yearly_ex_ret"].append( 
            52 * (return_df[pf].iloc[i*12*4:(i+1)*12*4].mean()) - 52 * (return_sp["mu"].iloc[i*12*4:(i+1)*12*4].mean())
        )

    # Calcul de l'ENC
    dic[pf]["enc"] = dic_weight[pf].apply(compute_enc, axis=1)
    
    # Information ratio
    dic[pf]["ir"] = compute_ir(dic[pf]["ex_ret"], dic[pf]["track_err"])
        
    # Yearly Information ratio
    dic[pf]["yearly_ir"] = compute_ir(dic[pf]["yearly_ex_ret"],
                                             dic[pf]["yearly_track_err"])
    
    # Diversification index # récupérer les poids
    sum_di = 0
    for i in range(len(return_df)//12):
          sum_di += compute_di(
            weight=dic_weight[pf].iloc[i],
            stock=stock_ret.iloc[i*12:(i+1)*12],
            vol_p=return_df[pf].iloc[i*12:(i+1)*12].std())
            
    print(pf)
    print(sum_di)
    dic[pf]["di"] = sum_di / (len(return_df)//12)
    
         

In [ ]:
dic

In [ ]:
%store dic

In [ ]:
dic["ew"].keys()

In [ ]:
dic.keys()

In [ ]:
dic[k]["ret"]

In [ ]:
plt.figure()
plt.bar(dic.keys(),
    [dic[k]["ret"] for k in dic.keys()],
       align="center",
        width=0.4
   )

plt.title("Rendement par portefeuille", fontsize=15)
plt.ylabel("%", fontsize=15)
plt.ylim(0,22)

plt.xticks(fontsize=12)
plt.yticks(fontsize=10)

In [ ]:
fig, axs = plt.subplots(1, len(dic.keys())+1, sharex=True, 
                        sharey=True,
                        figsize=(9, 6),
                        gridspec_kw={"width_ratios": [3, 3, 3, 3, 3, 1]})

dic_color = {'increasing':'green', 'decreasing':'red', 'no trend':'blue'}


labels = list(dic.keys())

for i, col in enumerate(return_df.columns):
    axs[i].hist(return_df[col], bins=50, density=True)
    axs[i].set_xlabel("%")
    axs[i].set_title(col, fontsize=11, pad=0.5)
    axs[i].grid(True)
    

    
lax = axs[-1]
lax.axis('off')
# lines = [Line2D([0], [0], color=c, linewidth=3, linestyle='-') for c in dic_color.values()]
#lax.legend(lines, labels, loc=10, fontsize=9)

# ----------------

fig.suptitle("Distribution des rendements hebdomadaires (%)")
#fig.suptitle("From {} to {})".format(start, end))
#fig.tight_layout(rect=[0,0,1,0.98])

In [ ]:
plt.figure()
plt.bar(dic.keys(),
    [dic[k]["vol"] for k in dic.keys()],
       align="center",
        width=0.4)

plt.title("Ecart-Type par portefeuille", fontsize=15)
plt.ylabel("%", fontsize=15)

plt.xticks(fontsize=12)
plt.yticks(fontsize=10)

Distribution des rendements


## Tracking error

In [ ]:
yr_idx = [str(i) for i in range(2001,2020)]

track_error_df = pd.DataFrame(index=yr_idx)

for k in dic.keys():
    track_error_df[k] = dic[k]["yearly_track_err"]
    
    

track_error_df.plot()    
plt.title("Tracking Error calculée annuellement")

In [ ]:
fig, axs = plt.subplots(1, len(dic.keys())+1, sharex=True, 
                        sharey=True,
                        figsize=(9, 6),
                        gridspec_kw={"width_ratios": [3, 3, 3, 3, 3, 1]})

dic_color = {'increasing':'green', 'decreasing':'red', 'no trend':'blue'}


labels = list(dic.keys())

for i, col in enumerate(return_df.columns):
    axs[i].hist(dic[col]["track_err"], bins=15)
    axs[i].set_xlabel("%")
    axs[i].set_title(f"{col}", fontsize=11, pad=0.5)
    axs[i].grid(True)
    

    
lax = axs[-1]
lax.axis('off')
# lines = [Line2D([0], [0], color=c, linewidth=3, linestyle='-') for c in dic_color.values()]
#lax.legend(lines, labels, loc=10, fontsize=9)

# ----------------

fig.suptitle("Distribution de la tracking error calculée trimestriellement (%)")
#fig.suptitle("From {} to {})".format(start, end))
#fig.tight_layout(rect=[0,0,1,0.98])

excess return

In [ ]:
yr_idx = [str(i) for i in range(2001,2020)]

ex_ret_df = pd.DataFrame(index=yr_idx)

for k in dic.keys():
    ex_ret_df[k] = dic[k]["yearly_ex_ret"]
    
    

ex_ret_df.plot()    
plt.title("Excess Return  annuel")

In [ ]:
ex_ret_df

In [ ]:
ex_ret_df.style.background_gradient(cmap="Blues")

In [ ]:
ex_ret_df.mean()

In [ ]:
ex_ret_df.mean() / ex_ret_df.std()

In [ ]:
yr_idx = [str(i) for i in range(len(dic["ew"]["enc"]))]

enc_df = pd.DataFrame(index=yr_idx)

for k in list(dic.keys())[1:]:
    enc_df[k] = dic[k]["enc"].values
    
    

enc_df.plot()    
plt.title("Evolution Trimestirelle de l'ENC")
plt.xlabel("Trismestre")

Information ratio

In [ ]:
yr_idx = [str(i) for i in range(2001,2020)]

ir_df = pd.DataFrame(index=yr_idx)

for k in dic.keys():
    ir_df[k] = dic[k]["yearly_ir"]

ir_df.plot()    
plt.title("Information Ratio annuel")

In [ ]:
ir_df.style.background_gradient(cmap="Blues")

sharp ratio

In [ ]:
plt.figure()
plt.bar(dic.keys(),
    [dic[k]["sr"] for k in dic.keys()],
       align="center",
        width=0.4)

plt.title("Sharp Ratio par portefeuille", fontsize=15)
plt.ylabel("%", fontsize=15)

plt.xticks(fontsize=12)
plt.yticks(fontsize=10)

### analysis

- Récupérer table de poids
- Calculer poids moyen sur les 20 ans
- Analyser les 3 plus gros poids pour les 5 portefeuilles

In [ ]:
path = f"weight_{w}.csv"
df = pd.read_csv(PROC_DATA / path)
df.mean().sort_values(ascending=False)[1:4]


In [ ]:
# Itérer sur les 5 portefeuille

for w in dic.keys():
    plt.figure()    
    path = f"weight_{w}.csv"
    df = pd.read_csv(PROC_DATA / path)
    print(w.upper())
    print(df.mean().sort_values(ascending=False)[1:4], "\n")
    top_assets = df.mean().sort_values(ascending=False)[1:4].index.tolist()
    df.drop(columns='Unnamed: 0', inplace=True)
    
    
    plt.plot(df.loc[:, ~df.columns.isin([top_assets])], color="black", alpha=0.2)
    plt.plot(df[top_assets], color="salmon")
    plt.title(f" {w.upper()} : Evolution des poids dans le temps")
    plt.ylim(0,1.2)
    
    
#     plt.figure()    
#     plt.plot(stock_ret.loc[:, stock_ret.columns.isin([df.columns])], color="black", alpha=0.2)
#     plt.plot(stock_ret[top_assets], color="salmon")
#     plt.title(f" {w.upper()} : Evolution des rendements dans le temps")
#     # plt.ylim(0,1.2)
    
    
    
    
    
    


Spider plot

In [ ]:
kpi_list = ["ret", "enc", "vol", "ex_ret", "track_err"]



In [ ]:
import plotly.graph_objects as go



for k in dic.keys():
    values = [np.mean(dic[k][kpi]) for kpi in kpi_list]
    
    values[3] *= 10
    values[4] *= 10
    

    fig = go.Figure(data=go.Scatterpolar(
      r=values,
      theta=kpi_list,
      fill='toself'
    ))

    fig.update_layout(
      polar=dict(
        radialaxis=dict(
          visible=True
        ),
      ),
      showlegend=False
    )

    
    fig.show()